# Libraries

In [7]:
import ast
import gensim
import gensim.corpora as corpora 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import scipy.cluster.hierarchy as shc

from collections import Counter
from gensim.models import CoherenceModel
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors._nearest_centroid import NearestCentroid

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data

In [8]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220131_feature_matrix_no_bert_instagram.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))


/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,90,94,98,102,103,104) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# LDA Clustering

## Initializing Terms and Texts

In [10]:
dat_obesity = dat_obesity[dat_obesity['tokens'].notna()]

texts = [ast.literal_eval(x) for x in dat_obesity.tokens]

terms = [item for sublist in texts for item in sublist]
terms = list(set(terms)) #106269

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initial LDA Set-Up

In [11]:
# Create Dictionary
id2word = corpora.Dictionary(texts) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts] 

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Picking Best # of Topics

In [12]:
# number of topics
for i in range(2, 26):
    lda_model_loop = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i,
                                           random_state = 110295)

    coherence_model_lda_loop = CoherenceModel(model=lda_model_loop, 
                                         texts=texts, 
                                         dictionary=id2word, 
                                         coherence='c_v')
    
    coherence_lda_loop = coherence_model_lda_loop.get_coherence()                                       
    print(f"Coherence of {coherence_lda_loop} for {i} topics.")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence of 0.543156788833566 for 2 topics.
Coherence of 0.4690047270697668 for 3 topics.
Coherence of 0.4336298068933912 for 4 topics.
Coherence of 0.476470001602992 for 5 topics.
Coherence of 0.403251617975014 for 6 topics.
Coherence of 0.41179786632107124 for 7 topics.
Coherence of 0.42582087611457176 for 8 topics.
Coherence of 0.47237880701182966 for 9 topics.
Coherence of 0.46255677571154763 for 10 topics.
Coherence of 0.45547282281698015 for 11 topics.
Coherence of 0.504989331111913 for 12 topics.
Coherence of 0.5016138433872016 for 13 topics.
Coherence of 0.5040959469938325 for 14 topics.
Coherence of 0.5037314170377222 for 15 topics.
Coherence of 0.5048249272194736 for 16 topics.
Coherence of 0.5127168279891476 for 17 topics.
Coherence of 0.5098694882200215 for 18 topics.
Coherence of 0.5027563319450677 for 19 topics.
Coherence of 0.5040634769768892 for 20 topics.
Coherence of 0.5042941789931268 for 21 topics.
Coherence of 0.505919565280229 for 22 topics.
Coherence of 0.491712

### Visualization

In [13]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2,
                                           random_state = 110295)

vis = pyLDAvis.gensim_models.prepare(topic_model = lda_model, 
                       corpus = corpus, 
                       dictionary = id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Export

In [14]:
pyLDAvis.save_html(vis, '220131_lda_instagram.html')

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Exporting Clusters

In [15]:
topic_probabilities = []
[topic_probabilities.append(i) for i in lda_model[corpus]]

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [16]:
topic_probs_dict = {i: topic_probabilities[i] for i in range(0, len(topic_probabilities))}

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
test = pd.DataFrame.from_dict({k:dict(v) for k,v in topic_probs_dict.items()}, orient='index').reset_index()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
test.sort_values('index').to_csv(file_path + "220131_lda_topic_probabilities_instagram.csv")
test = test.sort_values('index')

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Bigrams

In [19]:
dat_obesity_test = dat_obesity.head()

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
def bigram(row):
    lst = ast.literal_eval(row['tokens'])
    return [(lst[x], lst[x+1].strip()) for x in range(len(lst)-1)]

dat_obesity["bigrams"] = dat_obesity.apply(lambda row: bigram(row), axis=1)

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
dat_obesity["bigrams"].to_csv(file_path + "220127_instagram_post_bigrams.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
test = pd.read_csv(file_path + "/220127_obesity_with_bigrams.csv")

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df2 = test.groupby('lda_topic_renamed').agg({'V2': 'sum'})

df3 = df2.V2.apply(lambda row: Counter(row)).to_frame()
df3.V2.apply(lambda row: list(row)[0:10])

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


lda_topic_renamed
Topic: 1     [[, (, ', o, b, e, s, ,,  , a]
Topic: 10    [[, ], (, ', i, e, l, t, s, ,]
Topic: 11    [[, (, ', n, e, w, ,,  , y, a]
Topic: 12    [[, (, ', c, u, s, h, i, o, n]
Topic: 13    [[, (, ', d, a, c, h, s, u, n]
Topic: 14    [[, ], (, ', k, n, o, w, ,,  ]
Topic: 2     [[, (, ', u, g, h, ,,  , b, i]
Topic: 3     [[, (, ', m, o, r, b, i, d, l]
Topic: 4     [[, (, ', l, a, t, e, s, ,,  ]
Topic: 5     [[, (, ', m, o, r, b, i, d, l]
Topic: 6     [[, (, ', u, n, b, e, l, i, v]
Topic: 7     [[, (, ', o, b, e, s, i, t, y]
Topic: 8     [[, (, ', c, o, u, l, d, ,,  ]
Topic: 9     [[, (, ', a, s, k, ,,  , j, i]
Name: V2, dtype: object